In [1]:
import socket

NB='test'
DESCRIPTION='test notebook'
HOST = socket.gethostname()
HOST

'DESKTOP-7KHT10A'

In [2]:
import warnings
import os
from pathlib import Path

import mlflow
from dataclasses import dataclass
from dataclasses import asdict

import pandas as pd
#import lightgbm as lgb
#import xgboost as xgb
from catboost import CatBoostClassifier, CatBoostRegressor, Pool
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, mean_squared_error, roc_auc_score, average_precision_score, mean_squared_log_error
from sklearn.preprocessing import LabelEncoder

#from my_utils import get_notebook_path

warnings.simplefilter('ignore')
pd.set_option('display.max_columns', 25)
pd.set_option('display.max_rows', 25)

In [3]:
ROOT_DIR = Path('../')
DATA_DIR = ROOT_DIR / Path('data')

In [4]:
class Config:
    N_LABEL = 24
    N_FOLD = 5
    RANDOM_SATE = 42
    LR = 0.15
    PATIENCE = 30
    EPOCH = 300
    BATCH_SIZE = 6
    SKIP_EVALUATE_NUM = 5

In [5]:
train_df = pd.read_csv(DATA_DIR / Path('titanic_train.csv'))
test_df = pd.read_csv(DATA_DIR / Path('titanic_test.csv'))

display(train_df.shape)
display(train_df.head(5))
display(test_df.shape)
display(test_df.head(5))

(891, 12)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


(418, 11)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


## feature and target

In [6]:
target = 'Survived'
del_columns = ['Survived', 'Name']

features = list(set(train_df.columns) - set(del_columns))
features

['Age',
 'Embarked',
 'Fare',
 'SibSp',
 'PassengerId',
 'Pclass',
 'Sex',
 'Ticket',
 'Cabin',
 'Parch']

## train lgbm

### metrix

In [7]:
def evaluation(true, pred):
    return roc_auc_score(true, pred)

### encording

In [8]:
# 数値でないカラムをLabelEncording
categoricals = ['Sex', 'Ticket', 'Cabin','Embarked']
features = list(set(features) | set(categoricals))
for c in categoricals:
    le = LabelEncoder()
    le.fit(train_df.append(test_df)[c].astype(str))
    train_df[c] = le.transform(train_df[c].astype(str))
    test_df[c] = le.transform(test_df[c].astype(str))

# Count encoding
count_enc = ['Sex', 'Ticket', 'Cabin', 'Embarked']
count_features = []
for c in count_enc:
    _count = train_df.append(test_df)[c].astype(str).value_counts().to_dict()
    train_df[f'{c}_count'] = train_df[c].astype(str).map(_count)
    test_df[f'{c}_count'] = test_df[c].astype(str).map(_count)
    count_features.append(f'{c}_count')
features = list(set(features) | set(count_features))

### train

In [9]:
params = {
    'learning_rate': 0.01, 
    'loss_function': 'Logloss',
    'verbose':100,
    'iterations':5000,
    'early_stopping_rounds':100
}

# TargetEncording Config
target_enc = ['Sex', 'Ticket', 'Cabin', 'Embarked'] # 集計対象カラム
target_enc_key = ['Survived'] # 集計するキー

oof_preds = np.zeros(len(train_df))
y_pred = np.zeros(len(test_df))
models = []
cv_scores = {}
skf = StratifiedKFold(n_splits=Config.N_FOLD, random_state=Config.RANDOM_SATE, shuffle=True)
for fold, (train_index, test_index) in enumerate(skf.split(train_df[features], train_df[target])):

    print(f'====== fold {fold} ======')

    # TrainとTestに分割
    x_train, x_val = train_df.copy().iloc[train_index][features], train_df.copy().iloc[test_index][features]
    y_train, y_val =  train_df.iloc[train_index][target], train_df.iloc[test_index][target]

    test = test_df[features]

    # Target Encoding
    if len(target_enc) > 0:
        for t in target_enc_key:
            for c in target_enc:
                x_train[f'{c}_target_enc_by_{t}'] = train_df.iloc[train_index][c].map(train_df.iloc[test_index].groupby(c)[t].mean().to_dict())
                x_val[f'{c}_target_enc_by_{t}'] = train_df.iloc[test_index][c].map(train_df.groupby(c)[t].mean().to_dict())
                test[f'{c}_target_enc_by_{t}'] = test_df[c].map(train_df.groupby(c)[t].mean().to_dict())
            
    train_features = x_train.columns.to_list()
    
    # create Dataset
    train_pool = Pool(x_train, y_train, cat_features=categoricals)
    valid_pool = Pool(x_val, y_val, cat_features=categoricals)
    test_pool = Pool(test, cat_features=categoricals) 
    
    # train
    model = CatBoostClassifier(**params)
    model.fit(train_pool, eval_set=valid_pool)
    models.append(model)

        fold_pred = model.predict(valid_pool, prediction_type='Probability')[:, 1]
    
    score = evaluation(y_val, fold_pred)
    cv_scores[f'cv{fold}'] = score

    oof_preds[test_index] = fold_pred

    y_pred += model.predict(test_pool, prediction_type='Probability')[:, 1] / Config.N_FOLD

    print(f'cv score is {score}')

oof_score = evaluation(train_df[target], oof_preds)
print(f'OOF score is {oof_score}')

====== fold 0 ======
0:	learn: 0.6868264	test: 0.6867816	best: 0.6867816 (0)	total: 182ms	remaining: 15m 12s
100:	learn: 0.4384151	test: 0.4407478	best: 0.4407478 (100)	total: 2.84s	remaining: 2m 17s
200:	learn: 0.3856800	test: 0.3787269	best: 0.3787269 (200)	total: 5.41s	remaining: 2m 9s
300:	learn: 0.3618074	test: 0.3562360	best: 0.3562360 (300)	total: 7.8s	remaining: 2m 1s
400:	learn: 0.3460043	test: 0.3446222	best: 0.3445303 (397)	total: 10.1s	remaining: 1m 56s
500:	learn: 0.3330316	test: 0.3368402	best: 0.3368402 (500)	total: 12.5s	remaining: 1m 51s
600:	learn: 0.3221421	test: 0.3332695	best: 0.3332657 (591)	total: 14.7s	remaining: 1m 47s
700:	learn: 0.3124658	test: 0.3310504	best: 0.3310425 (699)	total: 16.9s	remaining: 1m 43s
800:	learn: 0.3033950	test: 0.3290568	best: 0.3290568 (800)	total: 19.4s	remaining: 1m 41s
900:	learn: 0.2926190	test: 0.3263272	best: 0.3263146 (899)	total: 21.7s	remaining: 1m 38s
1000:	learn: 0.2810334	test: 0.3248712	best: 0.3248697 (999)	total: 24.1s	r

array([0.03856667, 0.9695962 , 0.7789588 , 0.88491783, 0.06006826,
       0.15980322, 0.21843704, 0.0352238 , 0.83849015, 0.81369549,
       0.90511207, 0.95064898, 0.13948179, 0.00741306, 0.59729674,
       0.84639421, 0.0462376 , 0.38903679, 0.56001315, 0.93585073,
       0.07982685, 0.20549301, 0.91684866, 0.55064861, 0.02178564,
       0.24511974, 0.23694551, 0.5284851 , 0.82635949, 0.14050628,
       0.20723325, 0.97624945, 0.8844692 , 0.0724483 , 0.17113783,
       0.18226202, 0.26522762, 0.13846626, 0.33704168, 0.8296286 ,
       0.29630086, 0.66786999, 0.07568493, 0.93691891, 0.84524664,
       0.05965677, 0.18431066, 0.79983299, 0.16771328, 0.36639037,
       0.02010036, 0.14637314, 0.95083842, 0.7868108 , 0.20846902,
       0.33542087, 0.90769215, 0.17492985, 0.9643296 , 0.01678155,
       0.16203817, 0.94890977, 0.28483238, 0.01189749, 0.16848655,
       0.60387348, 0.81893077, 0.05198812, 0.52795147, 0.09407789,
       0.13665981, 0.02543793, 0.09017707, 0.16399895, 0.34639